https://dev.twitter.com/rest/reference/get/followers/ids

Returns ids of the followers of a user specified by the required user_id or screen_name parameter.

1. You need to enter your API credentials to the key list.
2. Script outputs the resulting data to followers_ids_out.txt as a csv formatted file

In [1]:
from birdy.twitter import UserClient, BirdyException 
from time import sleep
import string

key = [
    ["Consumer Key", "Consumer Secret", "Access Token", "Access Token Secret"]
]

client = UserClient(key[0][0], key[0][1], key[0][2], key[0][3])

In [4]:
uid = 0
sname = ''

In [5]:
followerIDs = []

keyInd = 0
client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])

protec = False

while(True):
    try:
        response = client.api.followers.ids.get(user_id = uid, screen_name = sname, count=5000, cursor=-1)
        break
    except Exception as err:
        print(err.status_code)
        print(err)
        if err.status_code == 429:
            sleep(60)
            keyInd = (keyInd + 1)%len(key)
        else:
            protec = True
            break
        client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])
        
if protec:
    followerIDs.append([])
    print('protected!')
else:
    ncur = response.data['next_cursor']
    for s in response.data['ids']:
        followerIDs.append(s)

    while(ncur != 0):
        while(True):
            try:
                response = client.api.followers.ids.get(user_id = uid, screen_name = sname, count=5000, cursor=ncur)
                break
            except Exception as err:
                print(err.status_code)
                print(err)
                if err.status_code == 429:
                    sleep(60)
                    keyInd = (keyInd + 1)%len(key)
                else:
                    sleep(15)
                client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])

        ncur = response.data['next_cursor']
        for s in response.data['ids']:
            followerIDs.append(s)

In [8]:
filename = 'followers_ids_out.txt'
f = open(filename, 'w', encoding='utf-8')

f.write(str(followerIDs[0]))
for ids in followerIDs[1:]:
    f.write(',')
    f.write(str(ids))

f.close()